<a href="https://colab.research.google.com/github/darkwingpatil/Ml_hackethons/blob/main/Rag_model_for_PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install langchain-core
!pip install langchain-openai
!pip install langchain-community
!pip install chromadb
!pip install pypdf
!pip install transformers
!pip install datasets
!pip install sentence-transformers
!pip install  faiss-cpu
!pip install text-generation
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalli

In [2]:
import openai
import os
from datasets import load_dataset

In [3]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 12.7 MB/s eta 0:00:00


## LOAD PDF DATA

In [4]:
from langchain_community.document_loaders import PyPDFLoader,JSONLoader
# Load PDF
pdf_loaders = [
    # Duplicate documents on purpose
    PyPDFLoader("/content/darkmatters.pdf")
]

docs = []
for loader in pdf_loaders:
    docs.extend(loader.load())

## Split the documents into chunks


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from datasets import load_dataset
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [6]:
# Split
#from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [7]:
splits = text_splitter.split_documents(docs)
print(len(splits))
print(len(splits[0].page_content) )
splits[0].page_content

339
493


'PlanManagement(DarkMatter)\nOverview\nPlanmanagementdealswiththeorganizationandmanagementofusersforB2B\ncustomers.\nWhatWeOwn\nhttps://github.com/ps-dev/plan-management-ConnectyourGithubaccount\nhttps://github.com/ps-dev/ps-mui-ConnectyourGithubaccount\nhttps://github.com/ps-dev/plans-account-switcher-ConnectyourGithubaccount\nhttps://github.com/ps-dev/plans-org-structure-selection-ConnectyourGithub\naccount\nDocs PlanManagement(DarkMatter)Deﬁnitions\n PlanManagement(DarkMatter)PublicEngineeringDocs'

## Load from huggingFace dataset

In [35]:
loader = HuggingFaceDatasetLoader(
    'WingPatil/combined_PM_rag_model_dataset',
    'text'
)

documents  = loader.load()
data_chunks = text_splitter.split_documents(documents)


In [9]:
combined_chunks = data_chunks + splits

In [10]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})

<ipython-input-10-dba272bcfd94>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain_community.vectorstores import Chroma # Light-weight and in memory

In [12]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any


In [13]:
vectordb = Chroma.from_documents(
    documents=combined_chunks, # splits we created earlier
    embedding=embeddings,
    persist_directory=persist_directory, # save the directory
)

In [14]:
retriever = vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 7, "fetch_k":15})

In [ ]:
!pip install huggingface_hub
from huggingface_hub import login

# Log in to Hugging Face
  # Replace with your actual token
login(token="xyz")

In [16]:
from langchain_huggingface import HuggingFaceEndpoint

In [29]:
# from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI


llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens = 512,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,
)
# HuggingFaceH4/zephyr-7b-beta
# mistralai/Mistral-7B-Instruct-v0.2    too good
#  meta-llama/Llama-3.2-3B-Instruct

In [18]:
from langchain_core.prompts import PromptTemplate     #
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [19]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

In [20]:
from langchain_core.runnables import RunnableLambda

In [30]:
rag_chain= {"context":RunnableLambda(lambda x:x["question"])| retriever,
         "question": lambda x:x["question"]}|QA_PROMPT | llm |StrOutputParser()

In [33]:
# Question 1
try:
  response=rag_chain.invoke({"question" :"Hi team! The support staff has received an internal escalation from the CTO about the ASML plan and we need prompt assistance"})
  print(response)
except Exception as e:
  print(e)

 thanks for asking! The issue seems to be related to the ASML plan, which is a Business Enterprise plan that is currently only showing Starter level plan analytics. The plan's authorizations are not being updated correctly, resulting in the removal of hundreds of users, although many have been added back. The plan's license type was switched from Professional to another type on January 8th. The support staff needs prompt assistance to resolve this issue. 

Note: The answer is based on the provided context and may not be a comprehensive solution to the problem. It's recommended to investigate further to find the root cause and implement a permanent fix. 

Thanks for asking!
